In [ ]:
# coding=utf-8
# Copyright 2023 Frank Latos AC8P
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#
# Much appreciation to the Pymoo project for providing the optimization framework used herein:
#
# pymoo: Multi-objective Optimization in Python
# https://github.com/anyoptimization/pymoo
# https://pymoo.org/index.html
#


In [44]:
import numpy as np
from numba import njit
import plotly.graph_objects as go






#
# Simple solid object visualizer
#
#
class SolidObjVisualizer():

    def __init__(self, x,y,z, width=800, height=700) -> None:
        
        # Create some visual elements: sxy = 'ground level', sxz = mirroring plane (transparent)
        # To change colors, see https://plotly.com/python/builtin-colorscales/
        sxy = go.Surface(x=x, y=y, z=np.full((2,2),0), colorscale='Greens', surfacecolor=np.full((2,2),0.69), showscale=False,cmin=0,cmax=1, hoverinfo='skip')
        sxz = go.Mesh3d(x=(x[0],x[0],x[1],x[1]), y=(0,0,0,0), z=(z[0],z[1],z[1],z[0]), i=(0,0),j=(1,2),k=(2,3),color='lightpink', opacity=0.1, hoverinfo='skip')
    
        self.fig = go.Figure(data=[sxy, sxz])                        # Create the figure, with fixed visual elements

        # Set up appearance: size of viewer, range for each axis, etc.
        self.fig.update_layout(
            width=width, height=height, autosize=False,
            scene=dict(
                # 'eye' is your initial viewing angle
                camera=dict(up=dict(x=0,y=0,z=1),  eye=dict(x=1,y=.3,z=0.3)),
                # Relative display sizes of axes; this produces a square patch in xy plane, with z (elevation) half that length
                # aspectratio = dict( x=1, y=1, z=0.5 ),
                aspectratio = dict( x=1, y=1, z=1.0 ),
                aspectmode = 'manual',
                # Limits of each axis, in meters
                xaxis=dict(range=x, showbackground=False),
                yaxis=dict(range=y, showbackground=False),
                zaxis=dict(range=z, showbackground=False),
            ),
        )

    # Make visible
    def show(self):
        self.fig.show()

    # Add a solid object
    #   s, t    vertexes and triangles as np.arrays
    def add(self, s, t, color, text='', opacity=1.0):
        if not isinstance(s, list):
            s = [s]
            t = [t]
        for i in range(len(s)):
            self.fig.add_mesh3d(i=t[i][:,0], j=t[i][:,1], k=t[i][:,2], x=s[i][:,0], y=s[i][:,1], z=s[i][:,2], 
                color=color, opacity=opacity, hovertemplate=text+'<extra></extra>')


    #
    # Create a polygon in xz plane, centered at (0,0,0)
    #
    #  nv       number of vertices
    #  r        distance from any vertex to origin
    #
    # Returns matrix of points (x,y,z)
    @staticmethod
    @njit
    def poly_xz(nv, r):
        assert nv > 2
        m = np.empty((nv,3))
        for i in range(nv):
            m[i] = np.array((r*np.cos(2*np.pi*i/nv), 0, r*np.sin(2*np.pi*i/nv)))
        return m


    #
    # Create a solid object by extruding a polygon along the y axis
    #
    #   m           array of points, shape (#vertices, 3)
    #   y           length of extrusion 
    #   close_end   close the first or second ends of extrusion?
    #   scale       scale of second face relative to first
    #
    # Returns: 
    #   s           array of points for both faces, shape (#vertices*2, 3)
    #   t           list of triangles, shape (#vertices*2, 3)
    #   
    @staticmethod
    def extrude_y(m, y, close_end=(False,False), scale=1.0):
        n = m.shape[0]                      # Number of points in initial shape
        s = np.vstack((m, m*scale + np.array((0,y,0)), np.array([[0,0,0],[0,y,0]])))     # Add other end face of solid
        t = []                              # Triangles (indexes of points in 's')
        for i in range(n):
            t.append( (i, i+n, (i+1)%n) )      # First triangle for this face
            t.append( ((i+1)%n, i+n, (i+1)%n + n) )     # Second triangle for this face
            if close_end[0]:
                t.append( (i, (i+1)%n, 2*n) )
            if close_end[1]:
                t.append( (i+n, (i+1)%n + n, 2*n+1) )
        return s, np.array(t)


    #
    # Create a cone
    #
    #   m           base of cone (array of points in xz plane)
    #   y           height (along y axis)
    #
    @staticmethod
    def cone_y(m, y):
        n = m.shape[0]                      # Number of points in initial shape
        s = np.vstack((m, np.array([[0,y,0]])))  
        t = np.empty((n, 3))              # Triangles (indexes of points in 's')
        for i in range(n):
            t[i] = np.array((i, (i+1)%n, n))      # Triangle for this face
        return s, t

    # Mirror list of arrays across xz plane
    @staticmethod
    def mirror_xz(arrs):
        mxz = np.array([[1,-1,1]])
        if isinstance(arrs, list):
            return [ar * mxz for ar in arrs]
        return arrs * mxz

    # Mirror list of arrays through origin
    @staticmethod
    def mirror_origin(arrs):
        mxz = np.array([[-1,-1,-1]])
        if isinstance(arrs, list):
            return [ar * mxz for ar in arrs]
        return arrs * mxz

    #
    # Translate and rotate antenna elements  (either a single array or list of arrays)
    #
    @staticmethod
    def _rot(elem,arr):
        if isinstance(elem, list):
            return [el @ arr for el in elem]
        return elem @ arr
    @staticmethod
    def rot_x(elem,ang):
        return SolidObjVisualizer._rot(elem, np.array([[1,0,0],[0,np.cos(ang),np.sin(ang)],[0,-np.sin(ang),np.cos(ang)]]))
    @staticmethod
    def rot_y(elem,ang):
        return SolidObjVisualizer._rot(elem, np.array([[np.cos(ang),0,-np.sin(ang)],[0,1,0],[np.sin(ang),0,np.cos(ang)]]))
    @staticmethod
    def rot_z(elem,ang):
        return SolidObjVisualizer._rot(elem, np.array([[np.cos(ang),np.sin(ang),0],[-np.sin(ang),np.cos(ang),0],[0,0,1]]))
    @staticmethod
    def translate(elem, dxyz):
        rowarr = np.array([dxyz])
        if isinstance(elem, list):
            return [el + rowarr for el in elem]
        return elem + rowarr


    #
    # Yagi construction
    #

    # Construct element along y axis
    #   lens:   list of section lengths (m)
    #   trs:    list of tubing radii (m)
    # Returns:  list of arrays of points, list of arrays of triangles
    @staticmethod
    def yagi_el(lens, trs, mirror=True, nsides=9):
        ps = []                     # Points (list of arrays)
        ts = []                     # Triangles (list of arrays)
        y = 0                       # y pos
        for i in range(len(lens)):
            poly = SolidObjVisualizer.poly_xz(nsides, trs[i])
            s, t = SolidObjVisualizer.extrude_y(poly, lens[i], close_end=(False,True))
            s = SolidObjVisualizer.translate(s, (0,y,0))
            ps.append(s)
            ts.append(t)
            y += lens[i]
        if mirror:
            ps.extend( SolidObjVisualizer.mirror_xz(ps) )
            ts.extend(ts)
        return ps, ts
    
    # Construct multiple yagi elements
    #   lens:   list of lists of section lengths, e.g. [ [1.0,1.1,1.2], [1.1,1.2,1.3], ...]
    #   trs:    list of tubing radii, e.g. [ [0.03,0.02,0.01], [0.03,0.02,0.01], ...]
    #   xpos    list of x positions, e.g. [-1, 0, 1.2]
    #   zpos    z height (m)
    # Returns:  list of arrays of points, list of arrays of triangles
    @staticmethod
    def yagi_els(lens, trs, xpos, zpos):
        ps = []                     # Points (list of arrays)
        ts = []                     # Triangles (list of arrays)
        for i in range(len(lens)):
            p, t = SolidObjVisualizer.yagi_el(lens[i], trs[i])
            p = SolidObjVisualizer.translate(p, (xpos[i],0,zpos))
            ps.extend(p)
            ts.extend(t)
        return ps, ts
    
    # Square/rectangular boom for yagi
    #   xpos    start, end positions on x axis, e.g. (-1, 1.2)
    #   zpos    z height (m)
    #   dim     cross-sectional dimensions: (y dim, z dim)
    # Returns:  list of arrays of points, list of arrays of triangles
    @staticmethod
    def yagi_boom(lens, trs, xpos, zpos):



In [46]:
from necutil import in2m

vis1 = SolidObjVisualizer((-3,3), (-3,3), (27,33), width=800, height=700)

lens = in2m( np.array([[24,36,29], [26,40,30], [28,42,32] ]) )
trs = in2m( np.array([[1.0, 7/8, 3/4]]*3) )
xpos = [1,0,-1.5]

# s,t = vis1.yagi_el(lens, trs, mirror=True)
s,t = vis1.yagi_els(lens, trs, xpos, zpos=30)

# p = vis1.poly_xz(9, 1)
# s,t = vis1.cone_y(p, 12)
# # s,t = vis1.extrude_y(p, 10, scale=0.5)
# s = vis1.rot_z(s, np.pi/2)
# # s = vis1.rot_y(s, np.pi/4)
# s = vis1.translate(s, (0,0,1.5))

vis1.add(s, t, 'lightsteelblue', text='', opacity=1.0)
vis1.show()



In [22]:
vis2 = SolidObjVisualizer((-15,15), (-15,15), (0,22), width=800, height=700)

p = vis2.poly_xz(5, 1)
s,t = vis2.extrude_y(p, 10, scale=0.5)
s2 = vis2.mirror_xz(s)
ss = vis2.rot_z([s,s2], np.pi/20)
ss = vis2.translate(ss, (0,1,10))
for m in ss:
    vis2.add(m, t, 'lightsteelblue', text='', opacity=1.0)

# s = vis1.rot_y(s, np.pi/4)

vis2.show()



In [76]:
ss = []
ts = []
p = poly_xz(9, 1)
s,t = extrude_y(p, 3, close_end=(True,False), scale=1.0)
print(s)
print(t)
# s = translate(s, (0,0,10))
ss.append(s)
ts.append(t)

# p = poly_xz(9, 0.8)
# s,t = extrude_y(p, 3, scale=1.0)
# s = translate(s, (0,3,10))
# ss.append(s)
# ts.append(t)

# p = poly_xz(9, 0.6)
# s,t = extrude_y(p, 3, scale=1.0)
# s = translate(s, (0,6,10))
# ss.append(s)
# ts.append(t)

# solid_obj_visualize(ss, ts, ['lightsteelblue'], (-15,15), (-15,15), (0,22), name='',width=800, height=700)
fig = solid_obj_visualize((-15,15), (-15,15), (0,22), width=800, height=700)
solid_obj_add(fig, s, t, color='lightsteelblue', text='', opacity=1.0)
fig.show()



[[ 1.          0.          0.        ]
 [ 0.76604444  0.          0.64278761]
 [ 0.17364818  0.          0.98480775]
 [-0.5         0.          0.8660254 ]
 [-0.93969262  0.          0.34202014]
 [-0.93969262  0.         -0.34202014]
 [-0.5         0.         -0.8660254 ]
 [ 0.17364818  0.         -0.98480775]
 [ 0.76604444  0.         -0.64278761]
 [ 1.          3.          0.        ]
 [ 0.76604444  3.          0.64278761]
 [ 0.17364818  3.          0.98480775]
 [-0.5         3.          0.8660254 ]
 [-0.93969262  3.          0.34202014]
 [-0.93969262  3.         -0.34202014]
 [-0.5         3.         -0.8660254 ]
 [ 0.17364818  3.         -0.98480775]
 [ 0.76604444  3.         -0.64278761]
 [ 0.          0.          0.        ]
 [ 0.          3.          0.        ]]
[[ 0  9  1]
 [ 1  9 10]
 [ 0  1 18]
 [ 1 10  2]
 [ 2 10 11]
 [ 1  2 18]
 [ 2 11  3]
 [ 3 11 12]
 [ 2  3 18]
 [ 3 12  4]
 [ 4 12 13]
 [ 3  4 18]
 [ 4 13  5]
 [ 5 13 14]
 [ 4  5 18]
 [ 5 14  6]
 [ 6 14 15]
 [ 5  6 18]
 [ 

In [29]:
close_end=(False,True)
if close_end[0]:
    print(0)
if close_end[1]:
    print(1)


1


In [40]:
t

[array([[ 0,  9,  1],
        [ 1,  9, 10],
        [ 9, 10, 19],
        [ 1, 10,  2],
        [ 2, 10, 11],
        [10, 11, 19],
        [ 2, 11,  3],
        [ 3, 11, 12],
        [11, 12, 19],
        [ 3, 12,  4],
        [ 4, 12, 13],
        [12, 13, 19],
        [ 4, 13,  5],
        [ 5, 13, 14],
        [13, 14, 19],
        [ 5, 14,  6],
        [ 6, 14, 15],
        [14, 15, 19],
        [ 6, 15,  7],
        [ 7, 15, 16],
        [15, 16, 19],
        [ 7, 16,  8],
        [ 8, 16, 17],
        [16, 17, 19],
        [ 8, 17,  0],
        [ 0, 17,  9],
        [17,  9, 19]]),
 array([[ 0,  9,  1],
        [ 1,  9, 10],
        [ 9, 10, 19],
        [ 1, 10,  2],
        [ 2, 10, 11],
        [10, 11, 19],
        [ 2, 11,  3],
        [ 3, 11, 12],
        [11, 12, 19],
        [ 3, 12,  4],
        [ 4, 12, 13],
        [12, 13, 19],
        [ 4, 13,  5],
        [ 5, 13, 14],
        [13, 14, 19],
        [ 5, 14,  6],
        [ 6, 14, 15],
        [14, 15, 19],
        